In [3]:
import nltk
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn_crfsuite as crfsuite
from sklearn_crfsuite import metrics
import os
from pathlib import Path
import re
import pickle
import random


In [4]:
entidade = ['dor', 'no', 'braço', 'esquerdo','e','direito']

lista=list()
for i in range(1, len(entidade)+1):
    #lista.append(entidade[i-1])
    for j in range(len(entidade) - i + 1):
        lista.append(entidade[j:j + i])
        
lista

[['dor'],
 ['no'],
 ['braço'],
 ['esquerdo'],
 ['e'],
 ['direito'],
 ['dor', 'no'],
 ['no', 'braço'],
 ['braço', 'esquerdo'],
 ['esquerdo', 'e'],
 ['e', 'direito'],
 ['dor', 'no', 'braço'],
 ['no', 'braço', 'esquerdo'],
 ['braço', 'esquerdo', 'e'],
 ['esquerdo', 'e', 'direito'],
 ['dor', 'no', 'braço', 'esquerdo'],
 ['no', 'braço', 'esquerdo', 'e'],
 ['braço', 'esquerdo', 'e', 'direito'],
 ['dor', 'no', 'braço', 'esquerdo', 'e'],
 ['no', 'braço', 'esquerdo', 'e', 'direito'],
 ['dor', 'no', 'braço', 'esquerdo', 'e', 'direito']]

In [5]:
def getTiposEntidade():
    return ['Problema','Teste','Tratamento','Anatomia']

def replaceWhiteSpaces(str):
    return re.sub('\s{2,}',' ',str)

def save_obj(name, obj):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    print('Load obj em: ', 'obj/' + name + '.pkl')
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [6]:
dic_sentencesTrain = load_obj(r'../spanclassification/obj/dic_sentencesTrain')
dic_sentencesDev = load_obj(r'../spanclassification/obj/dic_sentencesDev')
dic_sentencesTest = load_obj(r'../spanclassification/obj/dic_sentencesTest')


Load obj em:  obj/../spanclassification/obj/dic_sentencesTrain.pkl
Load obj em:  obj/../spanclassification/obj/dic_sentencesDev.pkl
Load obj em:  obj/../spanclassification/obj/dic_sentencesTest.pkl


In [17]:
dic_sentencesTest[1]

[[['Em', 0, 59],
  ['acompanhamento', 1, 62],
  ['no', 2, 77],
  ['ambualtorio', 3, 80],
  ['há', 4, 92],
  ['5', 5, 95],
  ['anos', 6, 97],
  ['por', 7, 102],
  ['FA', 8, 106],
  [',', 9, 108],
  ['uso', 10, 110],
  ['de', 11, 114],
  ['marevan', 12, 117],
  ['5mg', 13, 125],
  ['1', 14, 129],
  ['x', 15, 131],
  ['ao', 16, 133],
  ['dia', 17, 136],
  ['.', 18, 139]],
 [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]]

In [25]:
# formato:
lista=list()
numJanela=4
for key, value in dic_sentencesTest.items():
    #print('value:', value)
    entidades = value[1]
    if len(entidades)==0:
        continue
    for entidade in entidades:
        #print('entidade:', entidade)
        indiceEntidade1=entidade[1][0]
        indiceEntidade2=entidade[1][-1]
        vizinhosAntes = list()
        vizinhosDepois = list()
        #print('indiceEntidade:', indiceEntidade)
        for tokens in value[0]:
            indice=tokens[1]
            #print('token: {}, indice: {}'.format(tokens[0], indice))
            if indice+4>=indiceEntidade1 and indice<indiceEntidade1:
                vizinhosAntes.append(tokens[0])
            if indice-4<=indiceEntidade2 and indice>indiceEntidade2:
                vizinhosDepois.append(tokens[0])
        lista.append([entidade[0], vizinhosAntes, vizinhosDepois])
    break
lista

[['FA', ['há', '5', 'anos', 'por'], [',', 'uso', 'de', 'marevan']],
 ['marevan 5mg', ['FA', ',', 'uso', 'de'], ['1', 'x', 'ao', 'dia']]]

In [32]:
'''
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

powerset([1,2,3])
'''

'\nfrom itertools import chain, combinations\n\ndef powerset(iterable):\n    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"\n    s = list(iterable)\n    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))\n\npowerset([1,2,3])\n'

In [39]:
dic_predictions_results_ner = load_obj(r'../spanclassification/obj/dic_predictions_results_ner_800')

Load obj em:  obj/../spanclassification/obj/dic_predictions_results_ner_800.pkl


In [46]:
dic_predictions_results_ner[42]

[[['Paciente', 0],
  ['em', 1],
  ['tratamento', 2],
  ['para', 3],
  ['ICC', 4],
  ['diastólica', 5],
  ['há', 6],
  ['cerca', 7],
  ['de', 8],
  ['5', 9],
  ['a', 10],
  ['com', 11],
  ['melhora', 12],
  ['importante', 13],
  ['dos', 14],
  ['sintomas', 15],
  ['após', 16],
  ['início', 17],
  ['do', 18],
  ['tratamento', 19],
  ['.', 20]],
 [['ICC diastólica', [4, 5], 'Problema'], ['tratamento', [19], 'Tratamento']]]

In [86]:
# para corpus GENIA, ideal pq pega as descontinuas
# no nestedclin, as descontinuas
def powersetGenia(s):
    x = len(s)
    masks = [1 << i for i in range(x)]
    for i in range(1 << x):
        yield [ss for mask, ss in zip(masks, s) if i & mask]
        
def powerset(entidade):    
    lista=list()
    for i in range(1, len(entidade)+1):
        #lista.append(entidade[i-1])
        for j in range(len(entidade) - i + 1):
            lista.append(entidade[j:j + i])
    return lista
        
#print(list(powerset([4, 5, 6])))
print(powerset(['dor', 'no', 'peito']))


[['dor'], ['no'], ['peito'], ['dor', 'no'], ['no', 'peito'], ['dor', 'no', 'peito']]


In [22]:
#samplecorpusformat = [[['POI', 'B-Abbreviation'],  ['DE', 'O'],  ['LAVAGEM', 'O'],  ['+', 'O'],  ['CURETA', 'O'],  ['DE', 'O'],  ['TECIDO', 'O'],  ['NECROTICO', 'O'],  ['.', 'O']], [['18', 'O'],  [':', 'O'],  ['00', 'O'],  [':', 'O'],  ['PACIENTE', 'O'],  ['RETORNOU', 'O'],  ['DO', 'O'],  ['CC', 'B-Abbreviation'],  ['LUCIDO', 'O'], [',', 'O'],  ['ORIENTADO', 'O'],  [',', 'O'],  ['COMUNICATIVO', 'O'],  [';', 'O'], ['MANTEM', 'O'],  ['AVP', 'B-Abbreviation'],  ['COM', 'O'],  ['STP', 'B-Abbreviation'], ['.', 'O']]]

# corpus:
# gravar os indices dos tokens
# para cada entidade, gerar a combinação possivel de tokens e pegar as features



In [27]:
def read_clusters(cluster_file):
    word2cluster = {}
    with open(cluster_file) as i:
        for line in i:
            word, cluster = line.strip().split('\t')
            word2cluster[word] = cluster
    return word2cluster


def word2features(sent, i, word2cluster):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'word.cluster=%s' % word2cluster[word.lower()] if word.lower() in word2cluster else "0",
        'postag=' + postag
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1
        ])
    else:
        features.append('BOS')

    if i > 1: 
        word2 = sent[i-2][0]
        postag2 = sent[i-2][1]
        features.extend([
            '-2:word.lower=' + word2.lower(),
            '-2:word.istitle=%s' % word2.istitle(),
            '-2:word.isupper=%s' % word2.isupper(),
            '-2:postag=' + postag2
        ])        

        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1
        ])
    else:
        features.append('EOS')

    if i < len(sent)-2:
        word2 = sent[i+2][0]
        postag2 = sent[i+2][1]
        features.extend([
            '+2:word.lower=' + word2.lower(),
            '+2:word.istitle=%s' % word2.istitle(),
            '+2:word.isupper=%s' % word2.isupper(),
            '+2:postag=' + postag2
        ])

        
    return features


def sent2features(sent, word2cluster):
    return [word2features(sent, i, word2cluster) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

word2cluster = read_clusters("data/embeddings/clusters_nl.tsv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/embeddings/clusters_nl.tsv'